In [2]:
# Dataset
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/numenta/NAB/master/data/realKnownCause/nyc_taxi.csv')

data['timestamp'] = pd.to_datetime(data['timestamp'])

data.head()

,timestamp,value
0,2014-07-01 00:00:00,10844
1,2014-07-01 00:30:00,8127
2,2014-07-01 01:00:00,6210
3,2014-07-01 01:30:00,4656
4,2014-07-01 02:00:00,3820


In [3]:
# create moving-averages
data['MA48'] = data['value'].rolling(48).mean()
data['MA336'] = data['value'].rolling(336).mean()

# plot 
import plotly.express as px
fig = px.line(data, x="timestamp", y=['value', 'MA48', 'MA336'], title='NYC Taxi Trips', template = 'plotly_dark')
fig.show()

In [4]:
# Data preparation
# drop moving-average columns
data.drop(['MA48', 'MA336'], axis=1, inplace=True)

# set timestamp to index
data.set_index('timestamp', drop=True, inplace=True)

# resample timeseries to hourly 
data = data.resample('H').sum()

# creature features from date
data['day'] = [i.day for i in data.index]
data['day_name'] = [i.day_name() for i in data.index]
data['day_of_year'] = [i.dayofyear for i in data.index]
data['week_of_year'] = [i.weekofyear for i in data.index]
data['hour'] = [i.hour for i in data.index]
data['is_weekday'] = [i.isoweekday() for i in data.index]

data.head()

,value,day,day_name,day_of_year,week_of_year,hour,is_weekday
timestamp,,,,,,,
2014-07-01 00:00:00,18971,1,Tuesday,182,27,0,2
2014-07-01 01:00:00,10866,1,Tuesday,182,27,1,2
2014-07-01 02:00:00,6693,1,Tuesday,182,27,2,2
2014-07-01 03:00:00,4433,1,Tuesday,182,27,3,2
2014-07-01 04:00:00,4379,1,Tuesday,182,27,4,2


In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# --- Import all 12 models from PyOD ---
from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.cof import COF
from pyod.models.iforest import IForest
from pyod.models.hbos import HBOS
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.sod import SOD
from pyod.models.sos import SOS

# === 1. Load and Prepare Data ===
# (This setup is from our previous steps)
url = 'https://raw.githubusercontent.com/numenta/NAB/master/data/realKnownCause/nyc_taxi.csv'
data = pd.read_csv(url)
data['timestamp'] = pd.to_datetime(data['timestamp'])
data = data.set_index('timestamp')

# Create time-based features
data['day'] = data.index.day
data['day_name'] = data.index.day_name()
data['day_of_year'] = data.index.dayofyear
data['week_of_year'] = data.index.isocalendar().week
data['hour'] = data.index.hour
data['is_weekday'] = np.where(data.index.dayofweek < 5, 1, 0)

# === 2. Define Features and Scale Data ===
features = ['value', 'day', 'day_of_year', 'week_of_year', 'hour', 'is_weekday']
data_scaled = StandardScaler().fit_transform(data[features])

# === 3. Define the List of Models ===
# We create a dictionary where the key is the name
# and the value is the model's class.
models_to_run = {
    'ABOD': ABOD(contamination=0.1),
    'CBLOF': CBLOF(contamination=0.1, n_clusters=8, random_state=123),
    'COF': COF(contamination=0.1, n_neighbors=20),
    'IForest': IForest(contamination=0.1, random_state=123),
    'HBOS': HBOS(contamination=0.1, n_bins=10),
    'KNN': KNN(contamination=0.1, n_neighbors=5),
    'LOF': LOF(contamination=0.1, n_neighbors=20),
    'OCSVM': OCSVM(contamination=0.1, kernel='rbf'),
    'PCA': PCA(contamination=0.1, random_state=123),
    'MCD': MCD(contamination=0.1, random_state=123),
    'SOD': SOD(contamination=0.1, n_neighbors=20),
    'SOS': SOS(contamination=0.1, perplexity=30)
}

# === 4. Loop, Train, and Assign Results ===
for name, model in models_to_run.items():
    print(f"--- Running {name} ---")

    # Fit the model
    model.fit(data_scaled)

    # Get predictions and scores
    predictions = model.predict(data_scaled)
    scores = model.decision_function(data_scaled)

    # Assign results to new columns in the DataFrame
    data[f'Anomaly_{name}'] = predictions
    data[f'Score_{name}'] = scores

# === 5. Show the Final Result ===
print("\n--- All models complete. Final DataFrame: ---")
# This will show the original columns plus all the new Anomaly/Score columns
# print(data.head())
data.head()

# You can also check all the new columns that were created:
print("\nNew columns added:")
print([col for col in data.columns if 'Anomaly_' in col])

--- Running ABOD ---
--- Running CBLOF ---
--- Running COF ---
--- Running IForest ---
--- Running HBOS ---
--- Running KNN ---
--- Running LOF ---
--- Running OCSVM ---
--- Running PCA ---
--- Running MCD ---
--- Running SOD ---
--- Running SOS ---

--- All models complete. Final DataFrame: ---

New columns added:
['Anomaly_ABOD', 'Anomaly_CBLOF', 'Anomaly_COF', 'Anomaly_IForest', 'Anomaly_HBOS', 'Anomaly_KNN', 'Anomaly_LOF', 'Anomaly_OCSVM', 'Anomaly_PCA', 'Anomaly_MCD', 'Anomaly_SOD', 'Anomaly_SOS']


In [6]:
import plotly.graph_objects as go
import plotly.express as px

# Get the list of model names from the dictionary you created
# (If you don't have 'models_to_run', just manually list the names)
model_names = list(models_to_run.keys())
# e.g., model_names = ['ABOD', 'CBLOF', 'IForest', 'HBOS', 'KNN', 'LOF',
#                     'OCSVM', 'PCA', 'MCD', 'SOD', 'SOS']

# Create the base figure
fig = go.Figure()

# Add the main line for taxi trips
fig.add_trace(go.Scatter(
    x=data.index,
    y=data['value'],
    mode='lines',
    name='Taxi Trips (Value)'
))

# --- Add traces for *each* model's anomalies ---
# We'll set all but the first one to be invisible initially
for i, name in enumerate(model_names):
    anomaly_col = f'Anomaly_{name}'
    outlier_data = data[data[anomaly_col] == 1]

    fig.add_trace(go.Scatter(
        x=outlier_data.index,
        y=outlier_data['value'],
        mode='markers',
        name=f'{name} Anomaly',
        marker=dict(color='red', size=8),
        # Only make the first model visible by default
        visible=(i == 0)
    ))

# --- Create the dropdown menu ---
buttons = []
for i, name in enumerate(model_names):
    # Create a visibility list.
    # [True, False, False, ..., False]
    # The first 'True' is for the base 'value' line (trace 0)
    # The N traces after that are for the models.
    visibility = [True] + [False] * len(model_names)

    # Set the trace for the selected model to True
    visibility[i + 1] = True

    buttons.append(dict(
        label=name,
        method="update",
        args=[{"visible": visibility},
              {"title": f"NYC Taxi Trips - Anomalies by {name}"}]
    ))

# Add dropdown to the layout
fig.update_layout(
    updatemenus=[dict(
        active=0, # Sets the first model as the default
        buttons=buttons,
        direction="down",
        pad={"r": 10, "t": 10},
        showactive=True,
        x=0.1,
        xanchor="left",
        y=1.1,
        yanchor="top"
    )],
    title=f"NYC Taxi Trips - Anomalies by {model_names[0]}", # Initial title
    template='plotly_dark'
)

fig.show()

In [7]:
# Get the list of model names you used
# (Assumes 'models_to_run' dictionary still exists)
model_names = list(models_to_run.keys())

# Create an empty dictionary to store the new DataFrames
cleaned_datasets = {}

print(f"Original data shape: {data.shape}")

# Loop through each model name
for name in model_names:
    # Define the anomaly column for this model
    anomaly_col = f'Anomaly_{name}'

    # Filter the data, keeping only rows where Anomaly is 0 (normal)
    cleaned_df = data[data[anomaly_col] == 0].copy()

    # Store the cleaned DataFrame in the dictionary
    cleaned_datasets[name] = cleaned_df

    print(f"Dataset '{name}' (anomalies removed): {cleaned_df.shape}")



Original data shape: (10320, 31)
Dataset 'ABOD' (anomalies removed): (9388, 31)
Dataset 'CBLOF' (anomalies removed): (9288, 31)
Dataset 'COF' (anomalies removed): (9288, 31)
Dataset 'IForest' (anomalies removed): (9289, 31)
Dataset 'HBOS' (anomalies removed): (9313, 31)
Dataset 'KNN' (anomalies removed): (9742, 31)
Dataset 'LOF' (anomalies removed): (9454, 31)
Dataset 'OCSVM' (anomalies removed): (9288, 31)
Dataset 'PCA' (anomalies removed): (9288, 31)
Dataset 'MCD' (anomalies removed): (9288, 31)
Dataset 'SOD' (anomalies removed): (9297, 31)
Dataset 'SOS' (anomalies removed): (9288, 31)


In [8]:
# Loop through each key (model name) and value (DataFrame) in your dictionary
for name, df in cleaned_datasets.items():

    # Find all columns to drop in this specific DataFrame
    cols_to_drop = [col for col in df.columns if col.startswith('Anomaly_') or col.startswith('Score_')]

    # Drop the columns and update the DataFrame in the dictionary
    cleaned_datasets[name] = df.drop(columns=cols_to_drop)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import plotly.graph_objects as go
import pandas as pd # Added pandas to create a results table

# 1. Define your features (X) and target (y)
features = ['day', 'day_of_year', 'week_of_year', 'hour', 'is_weekday']
target = 'value'

# Create an empty list to store the results from each model
all_model_performance = []

# --- Loop through all 12 of your cleaned datasets ---
# 'name' will be 'IForest', 'LOF', etc.
# 'cleaned_df' will be the corresponding DataFrame
for name, cleaned_df in cleaned_datasets.items():

    print(f"\n--- Training Regression Model on data cleaned by: {name} ---")

    # 2. Get X and y from the *current* cleaned DataFrame
    X = cleaned_df[features]
    y = cleaned_df[target]

    # 3. Split the data chronologically
    split_point = int(len(X) * 0.8)
    X_train = X[:split_point]
    y_train = y[:split_point]
    X_test = X[split_point:]
    y_test = y[split_point:]

    print(f"Training data shape: {X_train.shape}, Testing data shape: {X_test.shape}")

    # 4. Create and train the model
    model = RandomForestRegressor(n_estimators=100, random_state=123, n_jobs=-1)

    print(f"Training model for {name}...")
    model.fit(X_train, y_train)
    print("Training complete.")

    # 5. Make predictions on the test set
    y_pred = model.predict(X_test)

    # 6. Calculate performance metrics
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"R-squared (R²): {r2:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.2f} trips")

    # 7. Store the results
    all_model_performance.append({
        'Cleaned_by_Model': name,
        'R2_Score': r2,
        'MAE': mae,
        'Data_Rows_Used': len(X)
    })

# --- After the loop, create a final comparison table ---
print("\n\n--- FINAL REGRESSION PERFORMANCE COMPARISON ---")
results_df = pd.DataFrame(all_model_performance)

# Sort by the best R-squared score
results_df = results_df.sort_values(by='R2_Score', ascending=False)

# Print the full results table
print(results_df.to_string())


--- Training Regression Model on data cleaned by: ABOD ---
Training data shape: (7510, 5), Testing data shape: (1878, 5)
Training model for ABOD...
Training complete.
R-squared (R²): 0.5723
Mean Absolute Error (MAE): 3475.14 trips

--- Training Regression Model on data cleaned by: CBLOF ---
Training data shape: (7430, 5), Testing data shape: (1858, 5)
Training model for CBLOF...
Training complete.
R-squared (R²): 0.5318
Mean Absolute Error (MAE): 3003.65 trips

--- Training Regression Model on data cleaned by: COF ---
Training data shape: (7430, 5), Testing data shape: (1858, 5)
Training model for COF...
Training complete.
R-squared (R²): 0.5285
Mean Absolute Error (MAE): 3659.74 trips

--- Training Regression Model on data cleaned by: IForest ---
Training data shape: (7431, 5), Testing data shape: (1858, 5)
Training model for IForest...
Training complete.
R-squared (R²): 0.7534
Mean Absolute Error (MAE): 2345.20 trips

--- Training Regression Model on data cleaned by: HBOS ---
Traini

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_absolute_error

# 🔹 Example: Suppose cleaned_datasets = { 'iforest': df1, 'lof': df2, 'svm': df3, 'knn': df4 }
# Each df must have the same columns: features + target

features = ['day', 'day_of_year', 'week_of_year', 'hour', 'is_weekday']
target = 'value'

# 🔹 Define the regression models to compare
regressors = {
    'dt': DecisionTreeRegressor(random_state=123),
    'rf': RandomForestRegressor(n_estimators=100, random_state=123, n_jobs=-1),
    'ada': AdaBoostRegressor(random_state=123),
    'gbc': GradientBoostingRegressor(random_state=123),
    'ridge': Ridge(),
    'lr': LinearRegression(),
    'svm': SVR(),
}

# 🔹 Initialize matrix to store R² results
results_matrix = pd.DataFrame(index=regressors.keys(), columns=cleaned_datasets.keys())

# 🔹 Loop through each cleaned dataset
for detector_name, df in cleaned_datasets.items():
    print(f"\n🚀 Evaluating regressors on data cleaned by: {detector_name.upper()}")

    X = df[features]
    y = df[target]

    # Chronological split (80/20)
    split_point = int(len(X) * 0.8)
    X_train, X_test = X[:split_point], X[split_point:]
    y_train, y_test = y[:split_point], y[split_point:]

    # Loop through each regression model
    for reg_name, reg in regressors.items():
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)
        r2 = r2_score(y_test, y_pred)

        results_matrix.loc[reg_name, detector_name] = round(r2, 4)

# 🔹 Print final R² score comparison table
print("\n=== 📊 FINAL R² SCORE MATRIX (Test Set Results) ===")
print(results_matrix)



🚀 Evaluating regressors on data cleaned by: ABOD

🚀 Evaluating regressors on data cleaned by: CBLOF

🚀 Evaluating regressors on data cleaned by: COF

🚀 Evaluating regressors on data cleaned by: IFOREST

🚀 Evaluating regressors on data cleaned by: HBOS

🚀 Evaluating regressors on data cleaned by: KNN

🚀 Evaluating regressors on data cleaned by: LOF

🚀 Evaluating regressors on data cleaned by: OCSVM

🚀 Evaluating regressors on data cleaned by: PCA

🚀 Evaluating regressors on data cleaned by: MCD

🚀 Evaluating regressors on data cleaned by: SOD

🚀 Evaluating regressors on data cleaned by: SOS

=== 📊 FINAL R² SCORE MATRIX (Test Set Results) ===
          ABOD   CBLOF      COF IForest     HBOS      KNN      LOF   OCSVM  \
dt      0.4334  0.4521   0.4028  0.4939   0.5265   0.3276   0.5084  0.5575   
rf      0.5723  0.5318   0.5285  0.7534   0.5931   0.5934   0.6538  0.6775   
ada     0.6065  0.6066    0.575  0.6469   0.5633   0.6333   0.6311  0.6261   
gbc     0.6144  0.5197   0.6038  0.758